In [1]:
import math
import metapy
import pytoml
import csv
import pandas as pd


class SearchEngine:
    def __init__(self, path, config):
        self.path = path
        self.idx = metapy.index.make_inverted_index(config)
        self.config = config
        self.query = metapy.index.Document()
        self.ranker = self.ranker(self.config)
        
    def build_corpus(self):
        df = pd.read_csv(self.path)
        textcolumn = ['segment_txt'] # ignore the other columns
        new_df_text = pd.DataFrame(df, columns=textcolumn)
        with open('course.dat', 'w') as f:
            for index, row in df.iterrows():
                text = row['segment_txt']
                f.write(text)
                f.write('\n')
        return df

    def ranker(self, config):
         return metapy.index.OkapiBM25(k1=1.65, b=0.75, k3=2.6)
        
    def ranker_score(self):
        ranker = self.ranker
        top_docs = ranker.score(self.idx, self.query, num_results=5)
        return top_docs

    def query_result(self,text):
        self.query.content(text) 
        top_docs = self.ranker_score()
        df = self.build_corpus()
        top_links = []
        for num, (d_id, _) in enumerate(top_docs):
            top_links.append(df.loc[d_id].segment_link)
        return top_links

ModuleNotFoundError: No module named 'metapy'